# Active Greedy Utility-based Agent

In [1]:
import gym 
import operator
import itertools
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import deque
from random import randint

In [2]:
env = gym.make('CartPole-v1')
env._max_episode_steps = 5000
ACTION_SPACE = env.action_space.n # number of possible actions
OBSERVATION_SPACE = env.observation_space.shape[0] # number of observable variables
EXPLORATION_PROB = 1.0 # EPSILON GREEDY STRATEGY
EXPLORATION_DECAY = 0.9995
EXPLORATION_STOP = 0.0001

In [3]:
def create_state_intervals():
    intervals = np.zeros((OBSERVATION_SPACE, STATES_IN_INTERVAL))
    intervals[0] = np.linspace(-4.8, 4.8, STATES_IN_INTERVAL)
    intervals[1] = np.linspace(-3.5, 3.5, STATES_IN_INTERVAL)
    intervals[2] = np.linspace(-0.42, 0.42, STATES_IN_INTERVAL)
    intervals[3] = np.linspace(-4, 4, STATES_IN_INTERVAL)
    return intervals

In [4]:
def discretize_observation(observation):
    discrete_observation = np.array([np.digitize(observation[index], INTERVALS[index])-1 for index in range(OBSERVATION_SPACE)])
    # if some value is under the lower border ignore it and give it min value
    discrete_observation = [0 if x<0 else x for x in discrete_observation]
    return discrete_observation

In [5]:
def get_all_possible_states():
    digits = len(str(STATES_IN_INTERVAL))
    state_indices = [str(state_index).zfill(digits) for state_index in range(STATES_IN_INTERVAL)] # all encodings for a single observation variable
    states = [state_indices for i in range(OBSERVATION_SPACE)] # for each observation variable a list of its encodings
    states = list(itertools.product(*states)) # get all permutation of all state encodings (->list of tuples)
    states = [''.join(x) for x in states] # join tuples to a single string
    return states

In [6]:
def observation_to_state(observation):
    discrete_observation = discretize_observation(observation)
    digits = len(str(STATES_IN_INTERVAL))
    
    state = ''
    for state_id in discrete_observation:
        if len(str(state_id)) < digits:
            state += str(state_id).zfill(digits)
        else:
            state += str(state_id)
    return state

In [7]:
env = gym.make('CartPole-v1')
env._max_episode_steps = 5000
ACTION_SPACE = env.action_space.n #number of possible actions
OBSERVATION_SPACE = env.observation_space.shape[0] #number of observable variables
STATES_IN_INTERVAL = 10
#LEARNING_RATE = 0.1
DISCOUNT_FACTOR = 0.99
EXPLORATION_PROB = 1.0 # EPSILON GREEDY STRATEGY
EXPLORATION_DECAY = 0.995
EXPLORATION_STOP = 0.01

In [8]:
def create_utility_table():
    states = get_all_possible_states()
    #utility_values = np.zeros(len(states))
    utility_values = np.empty(len(states))
    utility_values.fill(100)
    utility_table = dict(zip(states, utility_values))
    return utility_table

In [9]:
def create_reward_table():
    states = get_all_possible_states()
    rewards = np.zeros(len(states)) # init with zero; high; random
    reward_table = dict(zip(states, rewards))
    return reward_table

In [10]:
#create_utility_table()

In [11]:
# def update_number_state_action_table(nsa_table, state, action):
#     key = (state, action)
#     if key in nsa_table.keys():
#         nsa_table[key] += 1
#     else:
#         nsa_table[key] = 1

In [12]:
def update_number_state_action_next_state_table(nsas_table, state, action, next_state):
    if (state, action) in nsas_table.keys():
        if next_state in nsas_table[(state, action)].keys():
            nsas_table[(state, action)][next_state] += 1
        else:
            nsas_table[(state, action)][next_state] = 1
    else:
        nsas_table[(state, action)] = {}
        nsas_table[(state, action)][next_state] = 1
        
    return nsas_table

In [13]:
def get_transition_probability(nsas_table, state, action):
    next_states = nsas_table[(state, action)]
    temp = {}
    for next_state in next_states:
        temp[next_state] = nsas_table[(state, action)][next_state]/sum(nsas_table[(state, action)].values())
        
    return temp

In [14]:
def get_nsa(nsas_table, state, action):
    return sum(nsas_table[(state, action)].values())

In [15]:
def update_utility_estimate(utility_table, nsas_table, state, action, next_state, reward_table, epsilon, gamma):
    next_states = nsas_table[(state, action)].keys()
    u = 0
    probs = get_transition_probability(nsas_table, state, action)
    for next_state in next_states:
        u +=  probs[next_state] * utility_table[next_state] ##
    
    actions = [0, 1]
    f_values = []
    if (state, actions[0]) in nsas_table.keys():
        f_values.append(exploration_function(u, get_nsa(nsas_table, state, action), epsilon))
    if (state, actions[1]) in nsas_table.keys():
        f_values.append(exploration_function(u, get_nsa(nsas_table, state, action), epsilon))
    if not f_values:
        print('(O.O) we have a problem')
    
    utility_table[state] = reward_table[state] + gamma * max(f_values)
    return utility_table

In [16]:
def exploration_function(utility, n, epsilon):
    if n < epsilon:
        return 95
    else:
        return utility

In [17]:
def get_action(utility_table, nsas_table, state): 
    best_a0, best_a1 = -100, -100
    if (state, 0) in nsas_table.keys():
        next_states_a0 = nsas_table[(state, 0)].keys()
        best_a0 = max([utility_table[s] for s in next_states_a0])
    if (state, 1) in nsas_table.keys():
        next_states_a1 = nsas_table[(state, 1)].keys()
        best_a1 = max([utility_table[s] for s in next_states_a1])
    
    if best_a0 == best_a1:
        return randint(0,1)
    
    return (0 if best_a0 > best_a1 else 1)

In [18]:
INTERVALS = create_state_intervals()
number_of_games = 200000


#data = pd.DataFrame(columns = ['mean', 'max', 'solve_step'])
data_max = []
data_mean = []
data_solve = []

solved = False

for i in range(1):
    print('EPISODE:', i)
        # init things we dont understand
    utility_table = create_utility_table()
    nsas_table = {}
    reward_table = create_reward_table()

    gamma = 0.9
    explore = 1 ## change this (jahrers idea ... critical)
    epsilon = EXPLORATION_PROB
    
    if i != 0:
        data_max.append(game_max)
        data_mean.append(game_mean)
        if not solved:
            data_solve.append(-1)
    last100_rewards = deque(maxlen=100) # fifo queue
    game_max = []
    game_mean = []
    solved = False
    for game in range(number_of_games):
        overall_reward, done = 0, False
        observation = env.reset()
        state = observation_to_state(observation)

        while not done:
            #if game % 1000 == 0: env.render()
            #if np.random.rand() < epsilon:
            #    action = env.action_space.sample()
            #else:
            action = get_action(utility_table, nsas_table, state)

            observation, reward, done, _ = env.step(action)
            next_state = observation_to_state(observation)
            overall_reward += reward

            if done: reward = -500 # punish if agent dies

            reward_table[state] = reward
            nsas_table = update_number_state_action_next_state_table(nsas_table, state, action, next_state)

            utility_table = update_utility_estimate(utility_table, nsas_table, state, action, next_state, reward_table, explore, gamma)

            state = next_state

        if epsilon > EXPLORATION_STOP: epsilon *= EXPLORATION_DECAY

        if game % 100 == 0 and game != 0:
            print('Episode:', game, 'Epsilon:', round(epsilon, 3), 
                  'Mean-Reward:', np.mean(last100_rewards), 'Max-Reward:', max(last100_rewards))
        if (np.mean(last100_rewards) >= 195) and not solved: 
            print('TASK COMPLETED LAST 100 GAMES HAD AN AVERAGE SCORE >=195 ON GAME', game)
            print(last100_rewards)
            solved = True
            data_solve.append(game)
            
        if game % 100 == 0 and game != 0:
            game_max.append(max(last100_rewards))
            game_mean.append(np.mean(last100_rewards))
        
        last100_rewards.append(overall_reward) 
        


EPISODE: 0


C:\Users\bernhard\AppData\Local\Continuum\miniconda3\envs\ml\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\bernhard\AppData\Local\Continuum\miniconda3\envs\ml\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Episode: 100 Epsilon: 0.603 Mean-Reward: 77.79 Max-Reward: 498.0
Episode: 200 Epsilon: 0.365 Mean-Reward: 104.51 Max-Reward: 175.0
Episode: 300 Epsilon: 0.221 Mean-Reward: 107.93 Max-Reward: 205.0
Episode: 400 Epsilon: 0.134 Mean-Reward: 110.44 Max-Reward: 211.0
Episode: 500 Epsilon: 0.081 Mean-Reward: 106.22 Max-Reward: 184.0
Episode: 600 Epsilon: 0.049 Mean-Reward: 109.09 Max-Reward: 184.0
Episode: 700 Epsilon: 0.03 Mean-Reward: 108.77 Max-Reward: 209.0
Episode: 800 Epsilon: 0.018 Mean-Reward: 104.8 Max-Reward: 236.0
Episode: 900 Epsilon: 0.011 Mean-Reward: 112.12 Max-Reward: 399.0
Episode: 1000 Epsilon: 0.01 Mean-Reward: 111.5 Max-Reward: 339.0
Episode: 1100 Epsilon: 0.01 Mean-Reward: 109.06 Max-Reward: 338.0
Episode: 1200 Epsilon: 0.01 Mean-Reward: 113.95 Max-Reward: 222.0
Episode: 1300 Epsilon: 0.01 Mean-Reward: 118.2 Max-Reward: 369.0
Episode: 1400 Epsilon: 0.01 Mean-Reward: 103.89 Max-Reward: 280.0
Episode: 1500 Epsilon: 0.01 Mean-Reward: 105.86 Max-Reward: 292.0
Episode: 1600 E

Episode: 12700 Epsilon: 0.01 Mean-Reward: 78.86 Max-Reward: 122.0
Episode: 12800 Epsilon: 0.01 Mean-Reward: 96.41 Max-Reward: 134.0
Episode: 12900 Epsilon: 0.01 Mean-Reward: 94.78 Max-Reward: 116.0
Episode: 13000 Epsilon: 0.01 Mean-Reward: 94.88 Max-Reward: 121.0
Episode: 13100 Epsilon: 0.01 Mean-Reward: 95.29 Max-Reward: 125.0
Episode: 13200 Epsilon: 0.01 Mean-Reward: 95.77 Max-Reward: 129.0
Episode: 13300 Epsilon: 0.01 Mean-Reward: 95.13 Max-Reward: 199.0
Episode: 13400 Epsilon: 0.01 Mean-Reward: 95.74 Max-Reward: 127.0
Episode: 13500 Epsilon: 0.01 Mean-Reward: 96.2 Max-Reward: 127.0
Episode: 13600 Epsilon: 0.01 Mean-Reward: 94.96 Max-Reward: 121.0
Episode: 13700 Epsilon: 0.01 Mean-Reward: 95.32 Max-Reward: 117.0
Episode: 13800 Epsilon: 0.01 Mean-Reward: 97.84 Max-Reward: 132.0
Episode: 13900 Epsilon: 0.01 Mean-Reward: 96.44 Max-Reward: 132.0
Episode: 14000 Epsilon: 0.01 Mean-Reward: 96.21 Max-Reward: 134.0
Episode: 14100 Epsilon: 0.01 Mean-Reward: 94.92 Max-Reward: 121.0
Episode: 14

KeyboardInterrupt: 